In [ ]:
import pandas as pd
import prearrival_modul as pa

%load_ext autoreload
%autoreload 2

file_name = '\KCI Affiliate'
file_type = '.xlsx'
file_loc = r'C:\Users\fajar\Documents\Python\Data\Member Data\prearrival'


file_path = file_loc+file_name+file_type

In [ ]:
# mengecek seluruh nama sheets dan menampilkan semuanya dengan perulangan

xl = pd.ExcelFile(file_path)

sheets = xl.sheet_names  # see all sheet names

count = 0

for i in sheets:
    count += 1

    df_prearrival = pd.read_excel(file_path, sheet_name=i)
    
    df_prearrival['First Name'] = df_prearrival['First Name'].str.title()
    df_prearrival['LastName'] = df_prearrival['LastName'].str.title()

    df_prearrival['First Name'] = df_prearrival['First Name'].str.strip()
    df_prearrival['LastName'] = df_prearrival['LastName'].str.strip()

    df_prearrival['Arrival'] = pd.to_datetime(df_prearrival['Arrival'], format='%d/%m/%Y %H:%M:%S')
    df_prearrival['Departure'] = pd.to_datetime(df_prearrival['Departure'], format='%d/%m/%Y %H:%M:%S')

    df_prearrival['Arrival'] = df_prearrival['Arrival'].dt.strftime('%d/%m/%Y')
    df_prearrival['Departure'] = df_prearrival['Departure'].dt.strftime('%d/%m/%Y')

    df_prearrival.loc[df_prearrival['Resort Name'].isin(pa.get_resort()), 'Resort Exists'] = 'Exist'
    df_prearrival.loc[~df_prearrival['Resort Name'].isin(pa.get_resort()), 'Resort Exists'] = 'None'

    df_prearrival["Resort Name"] = df_prearrival.apply(
        lambda x: pa.location_correction(x['Resort Name']) ,
        axis=1,
    )

    df_prearrival.rename(columns={"Member #": "number", "Booking #": "booking_no", "Resort Name": "resort", "Arrival": "check_in", "Departure": "check_out"}, inplace=True)

    if (df_prearrival.shape[0]>5):
        new_sheet = i
    else:
        new_sheet = i+'(kecil)'

    new_file_path = file_loc+file_name+' - '+i+'.csv'

    # if (count == 1):
    #     df_prearrival.to_excel(new_file_path, sheet_name=new_sheet, index=False)

    # else:
    #     # jika ingin menambahkan sheets baru
    #     with pd.ExcelWriter(new_file_path, mode="a", engine="openpyxl") as writer:
    #         df_prearrival.to_excel(writer, sheet_name=new_sheet, index=False)

    df_prearrival.to_csv(new_file_path, index=False, encoding='ISO-8859-1')

    print('generate sheet '+i)
    # print(count)

